# WEB SCRAPING USING SELENIUM AND BEAUTIFULSOUP

In [1]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

# Extract page using selenium

In [3]:
driver = webdriver.Chrome(executable_path='\chromeDriver\chromedriver.exe')
driver.get('https://www.konga.com/category/accessories-computing-5227')

# Passing the extracted page to Beautifulsoup

In [4]:
page=bs(driver.page_source)

In [40]:
# page

In [6]:
products_section= page.find("ul", {"class" : "b49ee_2pjyI _58c31_2R34y _22339_3gQb9"})

In [7]:
product_info=products_section.find_all("li",{"class":"bbe45_3oExY _22339_3gQb9"})

In [8]:
# Number of item available on each page
len(product_info)

20

# Extract all the infornation needed from an instance of item

In [33]:
product_info[2].find("span",{"class":"_4472a_zYlL- _6c244_q2qap"}).text.split()[0]

'27%'

In [34]:
product_info[2].find("div",{"class":"af885_1iPzH"}).text

'Zinox Z100 Gtx Notebook - Intel Cherr...'

In [35]:
product_info[2].find("span",{"class":"d7c0f_sJAqi"}).text.strip()

'₦59,500'

In [36]:
product_info[2].find("span",{"class":"_7cc7b_23GsY"}).text.split()[-1]

'ashour-fzco'

In [17]:
product_info[2].find("span",{"class":"eea9b_1Ma8-"}).text

'(No reviews yet)'

In [18]:
product_info[2].find("span",{"class":"_17030_3ZCyn"}).text

#  Create a function to extract data on one page

In [19]:

def Get_data(product_info):
    store=[]
    products=product_info
    for info in products:
        infos={}
        try:
            infos['off %']=info.find("span",{"class":"_4472a_zYlL- _6c244_q2qap"}).text.split()[0]
        except:
            pass
        infos['Description']=info.find("div",{"class":"af885_1iPzH"}).text
        infos['Price']=info.find("span",{"class":"d7c0f_sJAqi"}).text.strip()
        try:  
            infos['number of review']=info.find("span",{"class":"eea9b_1Ma8-"}).text
            if infos['number of review']=='(No reviews yet)':
                infos['number of review']=0

        except:
            pass
        try:
            infos['Pick up']-info.find("div",{"class":"_09e22_1ojNd"}).text.split()[1:][0]
        except:
            pass
        try:
            infos['available at']=info.find("span",{"class":"_17030_3ZCyn"}).text
        except:
            pass
        store += [infos]
    return store

In [20]:
data1=Get_data(product_info)
pd.DataFrame((data1))

,off %,Description,Price,store name,number of review,available at
0,18%,HP Notebook 15-RB006NIA AMD A4 Dual C...,"₦88,500",konga,4 Reviews,"Lagos,Abuja,PH"
1,19%,HP Notebook 15 Intel Celeron N3060 - ...,"₦94,990",konga,1 Review,"Lagos,Abuja,PH"
2,27%,Zinox Z100 Gtx Notebook - Intel Cherr...,"₦59,500",ashour-fzco,0,Lagos
3,NaN,HP Stream 11 - Intel X5-e8000 - 4GB R...,"₦80,000",meeso,0,NaN
4,19%,HP 15 Laptop - Intel Core i3 - 1TB HD...,"₦157,000",inc,0,NaN
5,49%,Unique Rechargeable Wireless Mouse - ...,"₦3,500",ventures,0,Lagos
6,13%,HP Notebook 15-RB006NIA AMD A4 Dual C...,"₦89,500",konga,1 Review,Abuja
7,NaN,Laptop Stand & Cooling Pad,"₦3,300",tech,1 Review,NaN
8,20%,HP 15 Laptop- Intel Core i3- 8GB RAM-...,"₦159,000",digitalarena,0,NaN
9,NaN,"HP Pavilion X360-11.6""-intel Pentium-...","₦180,000",links,0,NaN


In [21]:
# get the first and last page number from a page
page1=int(page.find("ul",{"class":"_6d32f_2_vdu"}).text[0])
last_page=int(page.find("ul",{"class":"_6d32f_2_vdu"}).text[-2:])

# Data extraction of a particular section on all pages available

In [23]:
links=[f"https://www.konga.com/category/accessories-computing-5227?page={i}" for i in range(page1,last_page+1,1)]

def All_page_data(page_url):
    
    links=[f"{page_url}?page={i}" for i in range(page1,last_page+1,1)]
    data_store=[]
    for url in links:
        driver.get(url)
        page=bs(driver.page_source)
        products_section= page.find("ul", {"class" : "b49ee_2pjyI _58c31_2R34y _22339_3gQb9"})
        product_info=products_section.find_all("li",{"class":"bbe45_3oExY _22339_3gQb9"})
        data = get_data(product_info)
        data_store+=data
    return data_store

In [24]:
page1_data=All_page_data('https://www.konga.com/category/accessories-computing-5227?page=1')

In [25]:
pd.DataFrame(page1_data)

,off %,Description,Price,store name,number of review,available at
0,18%,HP Notebook 15-RB006NIA AMD A4 Dual C...,"₦88,500",konga,4 Reviews,"Lagos,Abuja,PH"
1,19%,HP Notebook 15 Intel Celeron N3060 - ...,"₦94,990",konga,1 Review,"Lagos,Abuja,PH"
2,27%,Zinox Z100 Gtx Notebook - Intel Cherr...,"₦59,500",ashour-fzco,0,Lagos
3,NaN,HP Stream 11 - Intel X5-e8000 - 4GB R...,"₦80,000",meeso,0,NaN
4,19%,HP 15 Laptop - Intel Core i3 - 1TB HD...,"₦157,000",inc,0,NaN
...,...,...,...,...,...,...
995,18%,HP Pavilion 15 Intel Core i5 - Touchs...,"₦295,000",ojaonline2,0,NaN
996,NaN,HP 26a Laserjet Toner Cartridge,"₦9,000",doublebigman,0,NaN
997,NaN,HP Pavilion 11 x360 Touch Screen - I...,"₦168,000",ltd,0,NaN
998,19%,HP 15 Laptop - Intel Core i3 - 1TB HD...,"₦159,000",gadgets,0,NaN


In [26]:
# https://www.konga.com/category/accessories-computing-5227
# https://www.konga.com/category/phones-tablets-5294
# https://www.konga.com/category/electronics-5261
# https://www.konga.com/category/konga-fashion-1259
# https://www.konga.com/category/home-kitchen-602
# https://www.konga.com/category/baby-kids-toys-8
# https://www.konga.com/category/other-categories-5971

# Get each page section link

In [27]:
div_links=page.find("div",{"class":"_0d327_3ftRW"})

In [28]:
qq=div_links.find_all('a')[1:]

In [29]:
" ".join(qq[0]['href'].split('/')[-1].split('-')[:-1])

'accessories computing'

In [30]:
section_url=[f"https://www.konga.com/category/{qq[i]['href'].split('/')[-1]}" for i in range(0,len(qq),1)]
section_url

['https://www.konga.com/category/accessories-computing-5227',
 'https://www.konga.com/category/phones-tablets-5294',
 'https://www.konga.com/category/electronics-5261',
 'https://www.konga.com/category/konga-fashion-1259',
 'https://www.konga.com/category/home-kitchen-602',
 'https://www.konga.com/category/baby-kids-toys-8',
 'https://www.konga.com/category/other-categories-5971']

# Get the data of all section through all the pages available

In [31]:
section_urls=[f"https://www.konga.com/category/{qq[i]['href'].split('/')[-1]}" for i in range(0,len(qq),1)]

for url in section_urls:
    section_data=[]
    data=all_page_data(url)
    section_data +=[data]
    section_name=" ".join(url.split('/')[-1].split('-')[:-1])
    print("\n RUNNING SECTION : "+section_name)
              
#   Store each section data into a csv file with the specific section name
    pd.concat([ pd.DataFrame(j) for j in section_data ]).to_csv( f"{section_name}.csv" ,index=False)    


 RUNNING SECTION : accessories computing

 RUNNING SECTION : phones tablets

 RUNNING SECTION : electronics

 RUNNING SECTION : konga fashion

 RUNNING SECTION : home kitchen

 RUNNING SECTION : baby kids toys

 RUNNING SECTION : other categories


In [39]:
# View one page section data
pd.read_csv("electronics.csv").head(20)

,off %,Description,Price,store name,number of review,available at
0,10%,"iTec 32"" Hd Led With Free Wall Bracke...","₦37,690",ashour-fzco,0,"Lagos,Abuja"
1,6%,"iTec 32"" HD LED with Free Wall Bracke...","₦39,500",ashour-fzco,0,Lagos
2,10%,Video Camera Camcorder Tripod with Ph...,"₦4,500",it,8 Reviews,NaN
3,50%,3.5mm Mini Studio Hands Free Micropho...,"₦2,000",it,21 Reviews,NaN
4,22%,Nsonic 24-inch Full HD LED TV + Free ...,"₦35,000",world,0,NaN
5,33%,"Samsung 32"" High Definition Flat TV","₦60,000",uchegold76,0,NaN
6,60%,Imagine 3 Pole 3.5mm Mini Studio Hand...,"₦2,000",it,0,NaN
7,44%,Lightweight Smartphone And Video Came...,"₦4,500",resources,5 Reviews,NaN
8,13%,7ft 2m Photo Studio Video Camera Ligh...,"₦7,000",it,1 Review,NaN
9,50%,Imagine 3 Pole 3.5mm Mini Studio Hand...,"₦1,500",it,0,NaN


In [32]:
driver.quit()